## One Hot Encoding - pandas

[Feature Engineering for Machine Learning Course](https://www.trainindata.com/p/feature-engineering-for-machine-learning)

One hot encoding, consists in encoding each categorical variable with different boolean variables (also called dummy variables) which take values 0 or 1, indicating if a category is present in an observation.

For example, for the categorical variable "Gender", with labels 'female' and 'male', we can generate the boolean variable "female", which takes 1 if the person is 'female' or 0 otherwise, or we can generate the variable "male", which takes 1 if the person is 'male' and 0 otherwise.

For the categorical variable "colour" with values 'red', 'blue' and 'green', we can create 3 new variables called "red", "blue" and "green". These variables will take the value 1, if the observation is of the said colour or 0 otherwise. 


### Encoding into k-1 dummy variables

Note however, that for the variable "colour", by creating 2 binary variables, say "red" and "blue", we already encode **ALL** the information:

- if the observation is red, it will be captured by the variable "red" (red = 1, blue = 0)
- if the observation is blue, it will be captured by the variable "blue" (red = 0, blue = 1)
- if the observation is green, it will be captured by the combination of "red" and "blue" (red = 0, blue = 0)

We do not need to add a third variable "green" to capture that the observation is green.

More generally, a categorical variable should be encoded by creating k-1 binary variables, where k is the number of distinct categories. In the case of gender, k=2 (male / female), therefore we need to create only 1 (k - 1 = 1) binary variable. In the case of colour, which has 3 different categories (k=3), we need to create 2 (k - 1 = 2) binary variables to capture all the information.

One hot encoding into k-1 binary variables takes into account that we can use 1 less dimension and still represent the whole information: if the observation is 0 in all the binary variables, then it must be 1 in the final (not present) binary variable.

**When one hot encoding categorical variables, we create k - 1 binary variables**


Most machine learning algorithms, consider the entire data set while being fit. Therefore, encoding categorical variables into k - 1 binary variables, is better, as it avoids introducing redundant information.


### Exception: One hot encoding into k dummy variables

There are a few occasions when it is better to encode variables into k dummy variables:

- when building tree based algorithms
- when doing feature selection by recursive algorithms
- when interested in determine the importance of each category

Tree based algorithms, as opposed to the majority of machine learning algorithms, **do not** evaluate the entire dataset while being trained. They randomly extract a subset of features from the data set at each node for each tree. Therefore, if we want a tree based algorithm to consider **all** the categories, we need to encode categorical variables into **k binary variables**.

If we are planning to do feature selection by recursive elimination (or addition), or if we want to evaluate the importance of each single category of the categorical variable, then we will also need the entire set of binary variables (k) to let the machine learning model select which ones have the most predictive power.


### Advantages of one hot encoding

- Straightforward to implement
- Makes no assumption about the distribution of the categorical variables
- Keeps all the information of the categorical variable
- Suitable for linear models

### Limitations

- Expands the feature space
- Does not add extra information while encoding
- Many dummy variables may be identical, introducing redundant information


### Notes

If our datasets contain a few highly cardinal variables, we will end up very soon with datasets with thousands of columns, which may make training of our algorithms slow, and model interpretation hard.

In addition, many of these dummy variables may be similar to each other, since it is not unusual that 2 or more variables share the same combinations of 1 and 0s. Therefore one hot encoding may introduce redundant or duplicated information even if we encode into k-1.


## In this demo:

We will see how to perform one hot encoding with pandas using the Titanic dataset.

For guidelines to obtain the data, check **section 2** of the course.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# load titanic dataset

usecols = ["pclass", "sibsp", "parch", "sex", "embarked", "cabin", "survived"]

data = pd.read_csv("../../Datasets/titanic.csv", usecols=usecols)

data.head()

,pclass,survived,sex,sibsp,parch,cabin,embarked
0,1,1,female,0,0,B5,S
1,1,1,male,1,2,C22,S
2,1,0,female,1,2,C22,S
3,1,0,male,1,2,C22,S
4,1,0,female,1,2,C22,S


In [3]:
# let's capture only the first letter of the
# cabin for this demonstration

data["cabin"] = data["cabin"].str[0]

data.head()

,pclass,survived,sex,sibsp,parch,cabin,embarked
0,1,1,female,0,0,B,S
1,1,1,male,1,2,C,S
2,1,0,female,1,2,C,S
3,1,0,male,1,2,C,S
4,1,0,female,1,2,C,S


### Encoding important

Just like imputation, all methods of categorical encoding should be performed over the training set, and then propagated to the test set. 

Why? 

Because these methods will "learn" patterns from the train data, and therefore you want to avoid leaking information and overfitting. But more importantly, because we don't know whether in future / live data, we will have all the categories present in the train data, or if there will be more or less categories. Therefore, we want to anticipate this uncertainty by setting the right processes right from the start. We want to create transformers that learn the categories from the train set, and used those learned categories to create the dummy variables in both train and test sets.

In [4]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("survived", axis=1),  # predictors
    data["survived"],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0,  # seed to ensure reproducibility
)

X_train.shape, X_test.shape

((916, 6), (393, 6))

### Let's explore the cardinality

In [5]:
# sex has 2 labels

X_train["sex"].unique()

array(['female', 'male'], dtype=object)

In [6]:
# embarked has 3 labels and missing data

X_train["embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [7]:
# cabin has 8 labels and missing data

X_train["cabin"].unique()

array([nan, 'E', 'C', 'D', 'B', 'A', 'F', 'T', 'G'], dtype=object)

## One hot encoding with pandas

### Advantages

- quick
- returns pandas dataframe
- returns feature names for the dummy variables

### Limitations of pandas:

- it does not preserve information from train data to propagate to test data


-----

The pandas method `get_dummies()`, will create as many binary variables as categories in the variable.

If the variable colour has 3 categories in the train data, it will create 2 dummy variables. However, if the variable colour has 5 categories in the test data, it will create 4 binary variables, therefore train and test sets will end up with different number of features and will be incompatible with training and scoring using Scikit-learn.

In practice, other implementations of one hot encoding are more useful than `get_dummies` when creating pipelines.

### into k  dummy variables

In [8]:
# we can create dummy variables with the build in
# pandas method get_dummies

tmp = pd.get_dummies(X_train["sex"])

tmp.head()

,female,male
501,True,False
588,True,False
402,True,False
1193,False,True
686,True,False


In [9]:
# for better visualisation let's put the dummies next
# to the original variable

pd.concat([X_train["sex"], pd.get_dummies(X_train["sex"])], axis=1).head()

,sex,female,male
501,female,True,False
588,female,True,False
402,female,True,False
1193,male,False,True
686,female,True,False


In [10]:
# and now let's repeat for embarked

tmp = pd.get_dummies(X_train["embarked"])

tmp.head()

,C,Q,S
501,False,False,True
588,False,False,True
402,True,False,False
1193,False,True,False
686,False,True,False


In [11]:
# for better visualisation

pd.concat([X_train["embarked"], pd.get_dummies(X_train["embarked"])], axis=1).head()

,embarked,C,Q,S
501,S,False,False,True
588,S,False,False,True
402,C,True,False,False
1193,Q,False,True,False
686,Q,False,True,False


In [12]:
# and now for cabin

tmp = pd.get_dummies(X_train["cabin"])

tmp.head()

,A,B,C,D,E,F,G,T
501,False,False,False,False,False,False,False,False
588,False,False,False,False,False,False,False,False
402,False,False,False,False,False,False,False,False
1193,False,False,False,False,False,False,False,False
686,False,False,False,False,False,False,False,False


In [13]:
# and now for all variables together: train set

# ========
# get_dummies automatically recognises variables of type
# object and categorical, ignoring numerical variables.
# ========

X_train_enc = pd.get_dummies(X_train)

print(X_train_enc.shape)

X_train_enc.head()

(916, 16)


,pclass,sibsp,parch,sex_female,sex_male,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T,embarked_C,embarked_Q,embarked_S
501,2,0,1,True,False,False,False,False,False,False,False,False,False,False,False,True
588,2,1,1,True,False,False,False,False,False,False,False,False,False,False,False,True
402,2,1,0,True,False,False,False,False,False,False,False,False,False,True,False,False
1193,3,0,0,False,True,False,False,False,False,False,False,False,False,False,True,False
686,3,0,0,True,False,False,False,False,False,False,False,False,False,False,True,False


In [14]:
# and now for all variables together: test set

X_test_enc = pd.get_dummies(X_test)

print(X_test_enc.shape)

X_test_enc.head()

(393, 15)


,pclass,sibsp,parch,sex_female,sex_male,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,embarked_C,embarked_Q,embarked_S
1139,3,0,0,False,True,False,False,False,False,False,False,False,False,False,True
533,2,0,1,True,False,False,False,False,False,False,False,False,False,False,True
459,2,1,0,False,True,False,False,False,False,False,False,False,False,False,True
1150,3,0,0,False,True,False,False,False,False,False,False,False,False,False,True
393,2,0,0,False,True,False,False,False,False,False,False,False,False,False,True


Notice the positives of pandas `get_dummies`:

- Dataframe returned with feature names.
- Automatically recognises variables of type object or categorical.
- Dummies appended to original data in place of the original categorical variables.

**And the limitations:**

The train set contains more dummy features than the test set. This occurred because there was no category T in cabin in the test set.

This will cause problems if training and scoring models with scikit-learn, because predictors require train and test sets to be of the same shape.

### into k -1 

In [15]:
# obtaining k-1 labels: we need to indicate get_dummies
# to drop the first binary variable

tmp = pd.get_dummies(X_train["sex"], drop_first=True)

tmp.head()

,male
501,False
588,False
402,False
1193,True
686,False


In [16]:
# obtaining k-1 labels: we need to indicate get_dummies
# to drop the first binary variable

tmp = pd.get_dummies(X_train["embarked"], drop_first=True)

tmp.head()

,Q,S
501,False,True
588,False,True
402,False,False
1193,True,False
686,True,False


For embarked, if an observation shows 0 for Q and S, then its value must be C, the remaining category.

Caveat, this variable has missing data, so unless we encode missing data as well, the dropped category and missing data will be treated equally.

In [17]:
# altogether: train set

X_train_enc = pd.get_dummies(X_train, drop_first=True)

print(X_train_enc.shape)

X_train_enc.head()

(916, 13)


,pclass,sibsp,parch,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T,embarked_Q,embarked_S
501,2,0,1,False,False,False,False,False,False,False,False,False,True
588,2,1,1,False,False,False,False,False,False,False,False,False,True
402,2,1,0,False,False,False,False,False,False,False,False,False,False
1193,3,0,0,True,False,False,False,False,False,False,False,True,False
686,3,0,0,False,False,False,False,False,False,False,False,True,False


In [18]:
# altogether: test set

X_test_enc = pd.get_dummies(X_test, drop_first=True)

print(X_test_enc.shape)

X_test_enc.head()

(393, 12)


,pclass,sibsp,parch,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,embarked_Q,embarked_S
1139,3,0,0,True,False,False,False,False,False,False,False,True
533,2,0,1,False,False,False,False,False,False,False,False,True
459,2,1,0,True,False,False,False,False,False,False,False,True
1150,3,0,0,True,False,False,False,False,False,False,False,True
393,2,0,0,True,False,False,False,False,False,False,False,True


### Bonus: get_dummies() can handle missing values

In [19]:
# we can add an additional dummy variable to indicate
# missing data

pd.get_dummies(X_train["embarked"], drop_first=True, dummy_na=True).head()

,Q,S,NaN
501,False,True,False
588,False,True,False
402,False,False,False
1193,True,False,False
686,True,False,False
